<!-- <center> Joaquin Peñuela-Parra </center>
<center> Department of Mechanical Engineering and Materials Science </center>
<center> University of Pittsburgh, Pittsburgh, Pennsylvania 15261, USA </center>  -->

$$ \textrm{Joaquin Penuela-Parra} $$
$$ \textrm{Department of Mechanical Engineering and Materials Science} $$
$$ \textrm{University of Pittsburgh, Pittsburgh, Pennsylvania 15261, USA} $$ 

In [1]:
using ITensors
using CUDA #Nvidia GPU

In [6]:
CUDA.devices()

CUDA.DeviceIterator() for 2 devices:
0. Tesla V100-SXM2-32GB
1. Tesla V100-SXM2-32GB

In [7]:
CUDA.memory_status()

Effective GPU memory usage: 0.95% (309.812 MiB/31.739 GiB)
Memory pool usage: 0 bytes (0 bytes reserved)


In general to use GPU with just need to use the function cu() or NDTensors.cu() to define the same ITensor Object inside the GPU Memory as a CUArray. The only difference between the two functions is that NDTensors.cu() preserves the element type of the tensors, while cu() converts to single precision. However, **single precision can generate problems in DMRG or TEBD** algorithms: https://itensor.discourse.group/t/tebd-with-gpu-error-with-eigen/1266/5 

**Block Sparse contraction**

Despite this type of contractions are still in development (https://itensor.discourse.group/t/ann-initial-release-of-new-itensor-gpu-backends/1227/3), we already can see some advantage of the use of GPU:

In [26]:
#woGPU
i = Index([QN(0) => 1000, QN(1) => 1000]);
A = randomITensor(i', dag(i));

@time (A)'*(A)

  0.041441 seconds (64 allocations: 30.526 MiB)


ITensor ord=2
(dim=2000|id=663)'' <Out>
 1: QN(0) => 1000
 2: QN(1) => 1000
(dim=2000|id=663) <In>
 1: QN(0) => 1000
 2: QN(1) => 1000
NDTensors.BlockSparse{Float64, Vector{Float64}, 2}

In [27]:
#wGPU
A = NDTensors.cu(A)

@time (A)'*(A)

  0.000311 seconds (196 allocations: 11.594 KiB)


ITensor ord=2
(dim=2000|id=663)'' <Out>
 1: QN(0) => 1000
 2: QN(1) => 1000
(dim=2000|id=663) <In>
 1: QN(0) => 1000
 2: QN(1) => 1000
NDTensors.BlockSparse{Float64, CuArray{Float64, 1, CUDA.DeviceMemory}, 2}

**Note:** If A is a CuArray and we define another variable in terms of A, that variable will be also a CuArray:

In [28]:
C = 2*A - 3*A

ITensor ord=2
(dim=2000|id=663)' <Out>
 1: QN(0) => 1000
 2: QN(1) => 1000
(dim=2000|id=663) <In>
 1: QN(0) => 1000
 2: QN(1) => 1000
NDTensors.BlockSparse{Float64, CuArray{Float64, 1, CUDA.DeviceMemory}, 2}

In [29]:
@time (C)'*(C)

  0.000294 seconds (196 allocations: 11.594 KiB)


ITensor ord=2
(dim=2000|id=663)'' <Out>
 1: QN(0) => 1000
 2: QN(1) => 1000
(dim=2000|id=663) <In>
 1: QN(0) => 1000
 2: QN(1) => 1000
NDTensors.BlockSparse{Float64, CuArray{Float64, 1, CUDA.DeviceMemory}, 2}

**Expectation values contractions (inner and apply functions)**

Pure states: $\langle A \rangle = \langle \Psi | A | \Psi \rangle$ 

In [30]:
#woGPU
sites = siteinds("S=1/2",50)
bond_dimension = 200 #The difference between times increases with this value.

A = randomMPS(sites, bond_dimension)
O = randomMPO(sites)

@time inner(A, apply(O, A))

  2.082095 seconds (66.47 k allocations: 933.940 MiB, 0.63% gc time)


-3.011871043689372e-16

In [31]:
#wGPU
A = NDTensors.cu(A)
O = NDTensors.cu(O)

@time inner(A, apply(O, A))

  0.287469 seconds (169.26 k allocations: 19.633 MiB)


-3.0118710440304213e-16

Mixed States: $\langle A \rangle = \textrm{Tr} (\rho A)$. Take for example when $\rho$ is an identity matrix, and A is a random operator.

In [32]:
#woGPU
ρ = MPO(sites, "Id")
O = randomMPO(sites)

@time tr(apply(ρ, O))

  0.019085 seconds (115.77 k allocations: 30.298 MiB)


1.960801096832753e-20

In [33]:
#wGPU
ρ = NDTensors.cu(ρ)
O = NDTensors.cu(O)

@time tr(apply(ρ, O))

#We also can perform the trace manually by contracting the bra and ket site indices of each site with delta tensors. This takes a similar time:
I = MPO(sites, "Id") #Contains all the delta tensors.
I = NDTensors.cu(I)

@time inner(I, apply(ρ, O))

  0.124633 seconds (277.64 k allocations: 34.710 MiB)
  0.158602 seconds (272.62 k allocations: 37.238 MiB, 11.88% gc time)


1.9608010968325657e-20

**Why it is not faster?**. I think that in this case we do not see advantage because the bond dimension of $ρ$ is 1. If we have an MPO with a bigger bond dimension, probably we will see the advantage as in the case of MPS.

In [34]:
ρ

MPO
[1] ((dim=2|id=274|"S=1/2,Site,n=1")', (dim=2|id=274|"S=1/2,Site,n=1"), (dim=1|id=80|"Link,l=1"))
[2] ((dim=2|id=372|"S=1/2,Site,n=2")', (dim=2|id=372|"S=1/2,Site,n=2"), (dim=1|id=242|"Link,l=2"), (dim=1|id=80|"Link,l=1"))
[3] ((dim=2|id=120|"S=1/2,Site,n=3")', (dim=2|id=120|"S=1/2,Site,n=3"), (dim=1|id=391|"Link,l=3"), (dim=1|id=242|"Link,l=2"))
[4] ((dim=2|id=491|"S=1/2,Site,n=4")', (dim=2|id=491|"S=1/2,Site,n=4"), (dim=1|id=673|"Link,l=4"), (dim=1|id=391|"Link,l=3"))
[5] ((dim=2|id=414|"S=1/2,Site,n=5")', (dim=2|id=414|"S=1/2,Site,n=5"), (dim=1|id=177|"Link,l=5"), (dim=1|id=673|"Link,l=4"))
[6] ((dim=2|id=818|"S=1/2,Site,n=6")', (dim=2|id=818|"S=1/2,Site,n=6"), (dim=1|id=551|"Link,l=6"), (dim=1|id=177|"Link,l=5"))
[7] ((dim=2|id=98|"S=1/2,Site,n=7")', (dim=2|id=98|"S=1/2,Site,n=7"), (dim=1|id=749|"Link,l=7"), (dim=1|id=551|"Link,l=6"))
[8] ((dim=2|id=906|"S=1/2,Site,n=8")', (dim=2|id=906|"S=1/2,Site,n=8"), (dim=1|id=577|"Link,l=8"), (dim=1|id=749|"Link,l=7"))
[9] ((dim=2|id=396|

**Garbage Collection** (references: https://cuda.juliagpu.org/stable/usage/memory/ and https://discourse.julialang.org/t/any-way-to-delete-an-object-and-free-memory/53600)

This can be monitored using the Task Manager of Windows or using CUDA.memory_status():

In [77]:
CUDA.memory_status()

Effective GPU memory usage: 6.58% (2.090 GiB/31.739 GiB)
Memory pool usage: 152.591 MiB (352.000 MiB reserved)


In [116]:
#woGPU
sites = siteinds("S=1/2",50)
bond_dimension = 600 #The difference between times increases with this value.

A = randomMPS(sites, bond_dimension)
O = randomMPO(sites)

@time begin 
    inner(A, apply(O, A))
end

 11.373794 seconds (66.92 k allocations: 7.421 GiB, 10.10% gc time)


-5.343098905360732e-16

In [117]:
#wGPU
A = NDTensors.cu(A)
O = NDTensors.cu(O)

@time inner(A, apply(O, A))

  0.824254 seconds (178.78 k allocations: 20.642 MiB)


-5.343098909471342e-16

As A has a bond_dimension of 600, the GPU used a lot of memory to perform the last operation:

In [118]:
CUDA.memory_status()

Effective GPU memory usage: 29.20% (9.267 GiB/31.739 GiB)
Memory pool usage: 7.050 GiB (8.812 GiB reserved)


<!-- ![Task Manager](CUDA_1.png)  -->

<!-- <img src="CUDA_1.png" alt="alt text" width="400" height="400"/> -->

If we try to run the code again, we must be careful because it could be slow because we do not have more GPU memory and ITensors has not clean all the cache.

In [136]:
for i = 1:10
    @time begin 
        inner(A, apply(O, A))
        CUDA.memory_status()    
        # CUDA.reclaim()
        # CUDA.memory_status()    
    end
end

Effective GPU memory usage: 54.11% (17.174 GiB/31.739 GiB)
Memory pool usage: 3.726 GiB (16.719 GiB reserved)
  0.809060 seconds (178.84 k allocations: 20.646 MiB, 1.14% gc time)
Effective GPU memory usage: 54.11% (17.174 GiB/31.739 GiB)
Memory pool usage: 10.411 GiB (16.719 GiB reserved)
  0.789810 seconds (178.96 k allocations: 20.657 MiB)
Effective GPU memory usage: 54.11% (17.174 GiB/31.739 GiB)
Memory pool usage: 2.052 GiB (16.719 GiB reserved)
  0.812577 seconds (178.97 k allocations: 20.657 MiB, 1.19% gc time)
Effective GPU memory usage: 54.11% (17.174 GiB/31.739 GiB)
Memory pool usage: 8.736 GiB (16.719 GiB reserved)
  0.788708 seconds (178.96 k allocations: 20.657 MiB)
Effective GPU memory usage: 54.11% (17.174 GiB/31.739 GiB)
Memory pool usage: 15.421 GiB (16.719 GiB reserved)
  0.793084 seconds (178.97 k allocations: 20.657 MiB)
Effective GPU memory usage: 54.50% (17.299 GiB/31.739 GiB)
Memory pool usage: 5.751 GiB (16.844 GiB reserved)
  0.800891 seconds (178.96 k allocatio

When we do not have more GPU the computer starts using the shared GPU memory (i.e. RAM), this generates a bottle neck and it could be even slower than just using CPU.

In [134]:
CUDA.memory_status()

Effective GPU memory usage: 54.11% (17.174 GiB/31.739 GiB)
Memory pool usage: 5.751 GiB (16.719 GiB reserved)


If we want to recover all the memory used to save the chache of the calculation we need to do the following:

In [132]:
@time GC.gc(true)
@time CUDA.reclaim()

  0.452791 seconds (98.26% gc time)
  0.044091 seconds (10 allocations: 608 bytes)


In [130]:
CUDA.memory_status()

Effective GPU memory usage: 19.84% (6.299 GiB/31.739 GiB)
Memory pool usage: 5.751 GiB (5.844 GiB reserved)


What if we clean in each step of the for?

In [41]:
for i = 1:8
    @time begin 
        inner(A, apply(O, A))
        CUDA.memory_status()    
    end
    @time begin
        GC.gc(true)
        CUDA.reclaim()
    end
end

Effective GPU memory usage: 22.98% (7.293 GiB/31.739 GiB)
Memory pool usage: 6.901 GiB (6.906 GiB reserved)
  0.796158 seconds (178.78 k allocations: 20.643 MiB)
  0.466342 seconds (10 allocations: 608 bytes, 93.54% gc time)
Effective GPU memory usage: 22.98% (7.293 GiB/31.739 GiB)
Memory pool usage: 6.901 GiB (6.906 GiB reserved)
  0.801689 seconds (178.77 k allocations: 20.643 MiB)
  0.492212 seconds (10 allocations: 608 bytes, 94.12% gc time)
Effective GPU memory usage: 22.98% (7.293 GiB/31.739 GiB)
Memory pool usage: 6.901 GiB (6.906 GiB reserved)
  0.814620 seconds (178.77 k allocations: 20.644 MiB)
  0.518652 seconds (10 allocations: 608 bytes, 93.98% gc time)
Effective GPU memory usage: 22.98% (7.293 GiB/31.739 GiB)
Memory pool usage: 6.901 GiB (6.906 GiB reserved)
  0.804223 seconds (178.77 k allocations: 20.643 MiB)
  0.480974 seconds (10 allocations: 608 bytes, 93.75% gc time)
Effective GPU memory usage: 22.98% (7.293 GiB/31.739 GiB)
Memory pool usage: 6.901 GiB (6.906 GiB re

In [44]:
CUDA.memory_status()

Effective GPU memory usage: 20.81% (1.664 GiB/7.996 GiB)
Memory pool usage: 481.346 MiB (512.000 MiB reserved)


It works, is better for the memory also takes some time.

**Multiple GPUs** https://cuda.juliagpu.org/stable/usage/multigpu/

In [60]:
#monitoring multiples gpu functions:

function memory_info_all_gpus(print_info = true)
    
    percentages = []

    scale = 1/(1024^3) #converty bytes to GB
    for (i, dev) in enumerate(CUDA.NVML.devices())

        name = CUDA.NVML.name(dev) 
        mem_info = CUDA.NVML.memory_info(dev)
        total = round(mem_info.total*scale, sigdigits=4)
        used = round(mem_info.used*scale, sigdigits=4)
        free = round(mem_info.free*scale, sigdigits=4)
        percentage= round(used*100/total, sigdigits=4)
        
        print_info ? println("$name #$i memory usage: $percentage % ($used GB/ $total GB)" ) : nothing
        
        append!(percentages, percentage)
    end
    
    return percentages
end

function clean_all_gpus()
    for i=reverse(0:length(CUDA.devices()) - 1)
        global current_gpu = i
        CUDA.device!(current_gpu)
        GC.gc(true) 
        CUDA.reclaim()
    end
end

clean_all_gpus (generic function with 1 method)

In [61]:
memory_info_all_gpus()

Tesla V100-SXM2-32GB #1 memory usage: 5.775 % (1.848 GB/ 32.0 GB)
Tesla V100-SXM2-32GB #2 memory usage: 1.76 % (0.5632 GB/ 32.0 GB)


2-element Vector{Any}:
 5.775
 1.76

In [65]:
sites = siteinds("S=1/2",50)
bond_dimension = 2000 #The difference between times increases with this value.

#wGPU
A = cu(randomMPS(sites, bond_dimension); unified=true)
O = cu(randomMPO(sites); unified=true)

memory_info_all_gpus()

Tesla V100-SXM2-32GB #1 memory usage: 7.894 % (2.526 GB/ 32.0 GB)
Tesla V100-SXM2-32GB #2 memory usage: 1.76 % (0.5632 GB/ 32.0 GB)


2-element Vector{Any}:
 7.894
 1.76

In [66]:
@time inner(A, apply(O, A))

 10.841058 seconds (247.56 k allocations: 23.779 MiB, 0.07% gc time)


4.0127992f-7

In [122]:
for i=1:5
    @time inner(A, apply(O, A))
    memory_info_all_gpus()
end

  4.889455 seconds (247.31 k allocations: 22.917 MiB)
 14.197200 seconds (247.53 k allocations: 22.930 MiB)
 19.496476 seconds (247.56 k allocations: 22.931 MiB)


In [67]:
memory_info_all_gpus()

Tesla V100-SXM2-32GB #1 memory usage: 20.74 % (6.637 GB/ 32.0 GB)
Tesla V100-SXM2-32GB #2 memory usage: 1.76 % (0.5632 GB/ 32.0 GB)


2-element Vector{Any}:
 20.74
  1.76

In [71]:
@time clean_all_gpus()
memory_info_all_gpus()

  0.879115 seconds (22 allocations: 1.219 KiB, 99.97% gc time)
Tesla V100-SXM2-32GB #1 memory usage: 7.344 % (2.35 GB/ 32.0 GB)
Tesla V100-SXM2-32GB #2 memory usage: 1.76 % (0.5632 GB/ 32.0 GB)


2-element Vector{Any}:
 7.344
 1.76

In [74]:
@time device!(0)
@time GC.gc(true) 
@time CUDA.reclaim()

  0.000008 seconds
  0.438500 seconds (99.98% gc time)
  0.000059 seconds (10 allocations: 608 bytes)


In [121]:
# for dev in devices()
#     # NOTE: normally you'd use events and wait for them
#     device!(dev)
#     synchronize()
# end

In [76]:
@time GC.gc()

  0.441875 seconds (99.98% gc time)


Unified is really important or not?

In [207]:
clean_all_gpus()
memory_info_all_gpus()

In [208]:
sites = siteinds("S=1/2",50)
bond_dimension = 1600 #The difference between times increases with this value.

#wGPU
device!(0)
# A = cu(randomMPS(sites, bond_dimension))
# O = cu(randomMPO(sites))

# A = cu(randomMPS(sites, bond_dimension); unified=true)
# O = cu(randomMPO(sites); unified=true)

A = NDTensors.cu(randomMPS(sites, bond_dimension))
O = NDTensors.cu(randomMPO(sites))

MPO
[1] ((dim=2|id=887|"S=1/2,Site,n=1")', (dim=2|id=887|"S=1/2,Site,n=1"), (dim=1|id=778|"Link,l=1"))
[2] ((dim=2|id=62|"S=1/2,Site,n=2")', (dim=2|id=62|"S=1/2,Site,n=2"), (dim=1|id=855|"Link,l=2"), (dim=1|id=778|"Link,l=1"))
[3] ((dim=2|id=128|"S=1/2,Site,n=3")', (dim=2|id=128|"S=1/2,Site,n=3"), (dim=1|id=207|"Link,l=3"), (dim=1|id=855|"Link,l=2"))
[4] ((dim=2|id=927|"S=1/2,Site,n=4")', (dim=2|id=927|"S=1/2,Site,n=4"), (dim=1|id=315|"Link,l=4"), (dim=1|id=207|"Link,l=3"))
[5] ((dim=2|id=312|"S=1/2,Site,n=5")', (dim=2|id=312|"S=1/2,Site,n=5"), (dim=1|id=909|"Link,l=5"), (dim=1|id=315|"Link,l=4"))
[6] ((dim=2|id=86|"S=1/2,Site,n=6")', (dim=2|id=86|"S=1/2,Site,n=6"), (dim=1|id=375|"Link,l=6"), (dim=1|id=909|"Link,l=5"))
[7] ((dim=2|id=706|"S=1/2,Site,n=7")', (dim=2|id=706|"S=1/2,Site,n=7"), (dim=1|id=964|"Link,l=7"), (dim=1|id=375|"Link,l=6"))
[8] ((dim=2|id=439|"S=1/2,Site,n=8")', (dim=2|id=439|"S=1/2,Site,n=8"), (dim=1|id=611|"Link,l=8"), (dim=1|id=964|"Link,l=7"))
[9] ((dim=2|id=449|

In [214]:
device!(1)
@time inner(A, apply(O, A))

  9.379785 seconds (180.83 k allocations: 22.561 MiB, 0.14% gc time)


2.590782771737994e-16

In [139]:
memory_info_all_gpus()

Tesla V100-SXM2-32GB #1 memory usage: 12.0 % (3.841 GB/ 32.0 GB)
Tesla V100-SXM2-32GB #2 memory usage: 1.76 % (0.5632 GB/ 32.0 GB)


2-element Vector{Any}:
 12.0
  1.76